In [1]:
# Importations
import sys      
import urllib
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup, Tag, NavigableString, Comment
import pickle
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer # ou CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from nltk.corpus import stopwords
from google.colab import drive
import tensorflow as tf
import pandas as pd

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

In [0]:
tf.enable_eager_execution()

In [3]:
#drive.mount('/content/gdrive')
#!ls '/content/gdrive/My Drive/Text_mining_projet'
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1mwnba7jZFOVy7YT0mCPYGnz7T_GOWYLU'
downloaded = drive.CreateFile({'id': '1mwnba7jZFOVy7YT0mCPYGnz7T_GOWYLU'})
downloaded.GetContentFile('OpenSubtitles.de-fr.fr')

KeyboardInterrupt: ignored

In [0]:
text = open("OpenSubtitles.de-fr.fr", "r").read() #.decode(encoding='utf-8')

In [5]:
print ('Length of text: {} characters'.format(len(text)))

Length of text: 558646528 characters


In [6]:
ncarac = int(len(text)/3)#int(len(text)/3)
ncarac

186215509

In [0]:
text = text[:ncarac]

In [0]:
#type(corpus_conv)

In [9]:
print(text[:250])

Bonjour. Je suis rédacteur au journal L'Illustration.
Je cherche M. Odilon Hubert, député d'Alger.
Il est là-bas.
Merci.
Monsieur Hubert ?
Camille Bruyère.
Vous êtes le 1er représentant des Français d'Algérie au Parlement.
Mon journal voudrait vous c


In [10]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

288 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
with open("idx2carCOURT", "wb") as fin :
  pickle.dump(idx2char, fin)

In [0]:
with open("car2idxCOURT", "wb") as fin :
  pickle.dump(char2idx, fin)

In [14]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  '+' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  ...
}


In [15]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Bonjour. Je s' ---- characters mapped to int ---- > [32 77 76 72 77 83 80 14  1 40 67  1 81]


In [16]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

B
o
n
j
o


In [17]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"Bonjour. Je suis rédacteur au journal L'Illustration.\nJe cherche M. Odilon Hubert, député d'Alger.\nIl"
" est là-bas.\nMerci.\nMonsieur Hubert ?\nCamille Bruyère.\nVous êtes le 1er représentant des Français d'A"
"lgérie au Parlement.\nMon journal voudrait vous consacrer un article.\nSur l'Algérie ?\nNon. Sur vous.\nS"
"ur moi ?\nC'est une histoire de violence et de sang, à la Shakespeare.\nLes lecteurs adorent ça, monsie"
"ur.\nVous croyez ?\nIl s'agit d'un autre pays, jeune homme, d'un autre monde, d'une autre terre où tout"


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [19]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "Bonjour. Je suis rédacteur au journal L'Illustration.\nJe cherche M. Odilon Hubert, député d'Alger.\nI"
Target data: "onjour. Je suis rédacteur au journal L'Illustration.\nJe cherche M. Odilon Hubert, député d'Alger.\nIl"


In [20]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 32 ('B')
  expected output: 77 ('o')
Step    1
  input: 77 ('o')
  expected output: 76 ('n')
Step    2
  input: 76 ('n')
  expected output: 72 ('j')
Step    3
  input: 72 ('j')
  expected output: 77 ('o')
Step    4
  input: 77 ('o')
  expected output: 83 ('u')


In [21]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 288) # (batch_size, sequence_length, vocab_size)


In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           73728     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 288)           295200    
Total params: 4,304,160
Trainable params: 4,304,160
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [28]:
sampled_indices

array([188, 273,  97, 218, 117, 186, 131,  57,  28, 241, 109,  40, 253,
        37,  19,  47, 189,  92, 117, 250, 147, 107, 235, 155, 273, 242,
       215,  15,  36, 260,  81, 149, 232,  31, 209,  80,  29, 121,  77,
       275, 112, 249, 225,  80,  12, 152,  39, 167,  57, 226,  87, 170,
       283, 118, 136,  44, 131, 252, 286, 110,  36,  98, 260, 135,  13,
       209,  11, 217, 262, 178, 220, 257,   2, 281, 251,  91,  51, 216,
        75, 225,  56, 152, 176, 137,  84, 202, 249, 226,  51, 102, 140,
       110, 263, 232, 205, 145, 229, 168,  83,  57])

In [29]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'uand nous marions-nous?\nBientôt.\nAussi tôt que tu voudras.\nDepuis trois générations, cette couronne '

Next Char Predictions: 
 'êⰰ\x86Œ\x9dè\xad[=Ρ\x94JξG3Që\x80\x9dι¿\x91ΕÇⰰΤŁ/F\u200esÁΑAÿr?¡oⴭ\x97θůr,ÄIÓ[űy×ﬀ\x9e³N\xadν\ufeff\x95F\x88\u200e²-ÿ+ń—àŔϋ!㤹κ~UłmůZÄÞ´vøθűU\x8c·\x95‚Αû½žÔu['


In [30]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 288)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.663828


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=5#5
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
   9523/Unknown - 1799s 189ms/step - loss: 1.2771

In [0]:
#model.save('model_caract.h5')

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"Bonjour"))

In [0]:
model.save('model_caractCOURT.h5')

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
filters += "'"
tokenizer = Tokenizer(filters=filters, oov_token='<UNK>')
tokenizer.fit_on_texts(corpus_conv)
#len(tokenizer.word_index)
tokenizer.word_index

{'<UNK>': 1,
 'je': 2,
 'vous': 3,
 'de': 4,
 'est': 5,
 'pas': 6,
 'le': 7,
 'la': 8,
 'il': 9,
 'que': 10,
 'un': 11,
 'à': 12,
 'ne': 13,
 'et': 14,
 'l': 15,
 'tu': 16,
 'a': 17,
 'c': 18,
 'ce': 19,
 'en': 20,
 'les': 21,
 'j': 22,
 'n': 23,
 'une': 24,
 'qu': 25,
 'd': 26,
 'ai': 27,
 'pour': 28,
 'on': 29,
 'ça': 30,
 'm': 31,
 'moi': 32,
 'nous': 33,
 'des': 34,
 'mais': 35,
 'qui': 36,
 'me': 37,
 'y': 38,
 'bien': 39,
 'si': 40,
 'elle': 41,
 'dans': 42,
 'du': 43,
 'au': 44,
 'tout': 45,
 'plus': 46,
 't': 47,
 'mon': 48,
 'suis': 49,
 'non': 50,
 'avec': 51,
 'oui': 52,
 's': 53,
 'votre': 54,
 'ils': 55,
 'te': 56,
 'être': 57,
 'lui': 58,
 'avez': 59,
 'fait': 60,
 'comme': 61,
 'va': 62,
 'se': 63,
 'ici': 64,
 'rien': 65,
 'faire': 66,
 'toi': 67,
 'là': 68,
 'sur': 69,
 'ma': 70,
 'était': 71,
 'as': 72,
 'où': 73,
 'peut': 74,
 'dit': 75,
 'pourquoi': 76,
 'cette': 77,
 'êtes': 78,
 'veux': 79,
 'sont': 80,
 'quand': 81,
 'sais': 82,
 'son': 83,
 'par': 84,
 'très': 8

In [0]:
#tokenizer.word_counts

In [0]:
char2idx = {u:i for i, u in enumerate(tokenizer.word_index)}
list(char2idx.keys())

['<UNK>',
 'je',
 'vous',
 'de',
 'est',
 'pas',
 'le',
 'la',
 'il',
 'que',
 'un',
 'à',
 'ne',
 'et',
 'l',
 'tu',
 'a',
 'c',
 'ce',
 'en',
 'les',
 'j',
 'n',
 'une',
 'qu',
 'd',
 'ai',
 'pour',
 'on',
 'ça',
 'm',
 'moi',
 'nous',
 'des',
 'mais',
 'qui',
 'me',
 'y',
 'bien',
 'si',
 'elle',
 'dans',
 'du',
 'au',
 'tout',
 'plus',
 't',
 'mon',
 'suis',
 'non',
 'avec',
 'oui',
 's',
 'votre',
 'ils',
 'te',
 'être',
 'lui',
 'avez',
 'fait',
 'comme',
 'va',
 'se',
 'ici',
 'rien',
 'faire',
 'toi',
 'là',
 'sur',
 'ma',
 'était',
 'as',
 'où',
 'peut',
 'dit',
 'pourquoi',
 'cette',
 'êtes',
 'veux',
 'sont',
 'quand',
 'sais',
 'son',
 'par',
 'très',
 'quoi',
 'dire',
 'vais',
 'voir',
 'allez',
 'comment',
 'es',
 'alors',
 'jamais',
 'aussi',
 'merci',
 'ou',
 'même',
 'sa',
 'peux',
 'chose',
 'ces',
 'homme',
 'deux',
 'tous',
 'faut',
 'bon',
 'ont',
 'encore',
 'autre',
 'ton',
 'avoir',
 'sans',
 'peu',
 'mes',
 'été',
 'toujours',
 'femme',
 'temps',
 'monsieur',
 

In [0]:
tokens = np.array(tokenizer.texts_to_sequences(corpus_conv))
tokens

array([list([179, 2, 49, 7577, 44, 696, 15, 20752]),
       list([2, 677, 31, 8462, 5714, 3283, 26, 5433]),
       list([9, 5, 68, 231]), ..., list([14, 16, 56, 7793, 1653]),
       list([35, 2, 13, 15, 146, 6]), list([221, 141])], dtype=object)

In [0]:
#idx2char = []
#for line in tokens :
#  for elt2 in line:
#    idx2char.append(elt2)
#idx2char[:10]

In [0]:
#char2idx

In [0]:
#vocab = []
#for line in corpus_conv :
#  word = line.split(' ')
#  for elt in word:
#    vocab.append(elt)
#vocab[:10]
vocab = list(char2idx.keys())
idx2char = np.array(vocab)
idx2char

array(['<UNK>', 'je', 'vous', ..., 'célimène', 'francen', 'casas'],
      dtype='<U27')

In [0]:
#text_as_int = np.array([char2idx[tokenizer.] for line in corpus_conv for c in line.split(' ')])
#text_as_int

#tokens = np.array(tokenizer.texts_to_sequences(corpus_conv))
#text_as_int = tokens
#text_as_int

In [0]:
text_as_int = []
for line in tokens :
  for elt2 in line:
    text_as_int.append(elt2)

text_as_int = np.array(text_as_int)
text_as_int[:10]

array([  179,     2,    49,  7577,    44,   696,    15, 20752,     2,
         677])

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text_as_int)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
#char_dataset.take(5)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

cet
vous
non
commencement
tout


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(' '.join(idx2char[item.numpy()])))

'cet vous non commencement tout retard tu religions vous déjeuner moi exagéré consacrer intelligente ai bâtir que pas sur soit ou vos consacrer parisiens fabriqué de veux la crâne endormi mais coin ai ment tout peuples suis retard venus de orient à frères ma tu ment avec ma de ma nous ce pas qu hier est régiment l est bureau ne il colons j hélie bataillon m vos de vont que votre aimes ai à ton côté chercher deux ai à ton oh ai qu ton chaque peut plus pas conquête la fille servis il faites jasmin êtes université est exotique'
'l est corses me de suffit ne il paradis désolé vous voir heure mais vend que votre aimes ai juste apôtre ai juste oran que votre aimes mais étonne homme naïve homme battait coiffer me une différence un j sûre l j certain ce pas quoi ont m es montre ne trop hélie êtes celle dignité est foutu coin en canal ariégeois maltais égaux oui mais disciplinaire mais 1837 mais enfoiré mais gratitude mais paysans mais étalon supporte plus est sa égypte très la interprète fais 

In [0]:
sequences.take(5)

<DatasetV1Adapter shapes: (101,), types: tf.int64>

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
dataset

<DatasetV1Adapter shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [0]:
dataset.take(1)

<DatasetV1Adapter shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(' '.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(' '.join(idx2char[target_example.numpy()])))

Input data:  'cet vous non commencement tout retard tu religions vous déjeuner moi exagéré consacrer intelligente ai bâtir que pas sur soit ou vos consacrer parisiens fabriqué de veux la crâne endormi mais coin ai ment tout peuples suis retard venus de orient à frères ma tu ment avec ma de ma nous ce pas qu hier est régiment l est bureau ne il colons j hélie bataillon m vos de vont que votre aimes ai à ton côté chercher deux ai à ton oh ai qu ton chaque peut plus pas conquête la fille servis il faites jasmin êtes université est'
Target data: 'vous non commencement tout retard tu religions vous déjeuner moi exagéré consacrer intelligente ai bâtir que pas sur soit ou vos consacrer parisiens fabriqué de veux la crâne endormi mais coin ai ment tout peuples suis retard venus de orient à frères ma tu ment avec ma de ma nous ce pas qu hier est régiment l est bureau ne il colons j hélie bataillon m vos de vont que votre aimes ai à ton côté chercher deux ai à ton oh ai qu ton chaque peut plus p

In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 179 ('cet')
  expected output: 2 ('vous')
Step    1
  input: 2 ('vous')
  expected output: 49 ('non')
Step    2
  input: 49 ('non')
  expected output: 7577 ('commencement')
Step    3
  input: 7577 ('commencement')
  expected output: 44 ('tout')
Step    4
  input: 44 ('tout')
  expected output: 696 ('retard')


In [0]:
# Batch size
BATCH_SIZE = 32

BUFFER_SIZE = 10000

# Créer les batchs d'entrainement
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<DatasetV1Adapter shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.keras.backend import set_session
tf.keras.backend.clear_session()  # For easy reset of notebook state.
config_proto = tf.ConfigProto()
off = rewriter_config_pb2.RewriterConfig.OFF
config_proto.graph_options.rewrite_options.arithmetic_optimization = off
session = tf.Session(config=config_proto)
set_session(session)
  
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [0]:
dataset.take(1)

<DatasetV1Adapter shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape) #, "# (batch_size, sequence_length, vocab_size)")

(32, 100, 59190)


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 256)           15152640  
_________________________________________________________________
gru (GRU)                    (32, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (32, None, 59190)         60669750  
Total params: 79,757,622
Trainable params: 79,757,622
Non-trainable params: 0
_________________________________________________________________


In [0]:
example_batch_predictions

<tf.Tensor: id=9032, shape=(32, 100, 59190), dtype=float32, numpy=
array([[[ 1.27310632e-03,  1.77357998e-03,  6.23482454e-04, ...,
         -1.27434637e-03, -2.12698802e-03,  7.49703031e-04],
        [ 1.70402462e-03,  1.72036677e-03,  1.51417102e-03, ...,
         -7.50317995e-04, -4.72564076e-04,  7.33080087e-04],
        [ 3.36846244e-03,  1.27291540e-03,  8.11273872e-04, ...,
          3.88385786e-04,  6.02740911e-04, -9.37095494e-04],
        ...,
        [ 4.39111900e-05,  1.17179639e-04,  2.36941874e-03, ...,
          1.17184070e-03,  1.35719107e-04, -2.70659104e-04],
        [ 3.71110829e-04, -1.27898908e-04,  1.76486821e-04, ...,
         -7.01611571e-04,  2.94057216e-04,  3.57456272e-04],
        [ 2.37392448e-03,  4.60190029e-04,  4.21760087e-05, ...,
          5.65779454e-04,  1.08331977e-03, -1.20022823e-03]],

       [[ 1.93910871e-03, -3.12645163e-04,  8.79561529e-04, ...,
         -8.28954915e-04, -9.26477136e-04, -7.13765039e-04],
        [ 1.83387636e-03, -3.0905727

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([54515, 26616,  7535, 49973,  1026, 37054, 46253,  3703, 25659,
       13740, 48013, 32982, 56658, 42614, 51033, 49934, 11085, 24489,
        9653, 27663, 52950, 48736, 35676, 41191, 36867, 35754, 49274,
         527, 19093, 11240, 58303, 27410, 31798, 52075, 28235,  6302,
        2961, 34675, 57650, 11111, 19904, 37367, 46511, 51223, 55293,
        6924, 15928, 18044,  6925, 10646,  3501,  2259, 43907, 58086,
        1524, 23899, 49405, 55870, 24217, 44664, 12348, 28311, 55699,
       37707, 45669,  4438,  3481, 51972, 23518,  9270, 35939, 27541,
       15542, 16550, 19218,   325, 48090, 58654, 28065, 33941, 39854,
       21704, 46877, 42276, 28326, 26139, 10291, 57081, 26071, 56709,
       18072, 10622, 17913,  4363,  9382, 52036, 35938, 28971, 48292,
       11363])

In [0]:
print("Input: \n", repr(" ".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr(" ".join(idx2char[sampled_indices ])))

Input: 
 'demandé moyen il matrones aime regarde votre que de empêche prêts adresse est nous qu pourrais temps accusations ne rasent est pense ne pense fait de sortez moyen gros de chez commun voulez sommes un de la voici part chance an part êtes temps c la important présente la t chercher domestique est tu évaluer et mot le outils oui qu merci adieu temps va ce pas ouverte n cela va tourmentent sais êtes temps c capable était femmes de patron de dr un trop sentie ne tu damnée est êtes temps différents peux dis les rencontré rend ma il'

Next Char Predictions: 
 'mensonger trompeuses rhett feuillards neuf deus rechargée suffisant chirurgiens 27ème spectrographe veillerait flânèrent outrepasse préciserai paysager brack fréquentais cachait franquiste amendez aménagements coucheras activais édito rasés dévoraient malade invoque gaucher déroula médor patricien agronomie insense post opération coûta échalotes bille traînait individual encroûte velu millionième gorges prétendants mécaniciens

Train the model

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (32, 100, 59190)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       10.988517


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

# Directory where the checkpoints will be saved
checkpoint_dir = '/content/gdrive/My Drive/training_checkpoints.hdf5'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
EPOCHS= 50 #10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
    917/Unknown - 428s 466ms/step - loss: 7.3024

InvalidArgumentError: ignored

Exportation du modèle

In [0]:
#pickle.dump(model, open('model_genere', 'wb'))

Lecture du modèle

In [0]:
#model_genere = pickle.load(open('model_genere', 'rb'))

Generate text

Restore the latest checkpoint
To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'/content/gdrive/My Drive/training_checkpoints.hdf5/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            6123264   
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 23919)          24516975  
Total params: 34,575,471
Trainable params: 34,575,471
Non-trainable params: 0
_________________________________________________________________


In [0]:
with open("tokenizer_corpus", "wb") as fin :
  pickle.dump(tokenizer, fin)

In [0]:
with open("idx2char", "wb") as fin :
  pickle.dump(idx2char, fin)

In [0]:
model.save('model.h5')

In [0]:
# import dill as pickle
# pickle.dump(model,open('model_genere',"wb"))

In [0]:
# pickle.load(open(model_genere, 'rb'))

In [0]:
# import h5py
# modele = h5py.File('model.h5', 'a')

In [0]:
id = '1cd6CDdBJkTPbFUTLFzRN9gyIFgKjsL3N'
downloaded = drive.CreateFile({'id': '1cd6CDdBJkTPbFUTLFzRN9gyIFgKjsL3N'})
downloaded.GetContentFile('model.h5')

In [0]:
import h5py
model = h5py.File('model.h5', 'r')

The prediction loop

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 20

  # Converting our start string to numbers (vectorizing)
  input_eval = tokenizer.texts_to_sequences(start_string) #[char2idx[s] for s in start_string]
  input_evale = [y for x in input_eval for y in x]
  input_eval = input_evale

  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1 # 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ' '.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"Bonjour, ça va ? "))

Bonjour, ça va ? arrives jamais me tu lune le pour bien se temps c peux rien je autres elle à gâché alors ça


In [0]:
# model = build_model(
#   vocab_size = len(vocab),
#   embedding_dim=embedding_dim,
#   rnn_units=rnn_units,
#   batch_size=BATCH_SIZE)

In [0]:
# optimizer = tf.keras.optimizers.Adam()

In [0]:
# @tf.function
# def train_step(inp, target):
#   with tf.GradientTape() as tape:
#     predictions = model(inp)
#     loss = tf.reduce_mean(
#         tf.keras.losses.sparse_categorical_crossentropy(
#             target, predictions, from_logits=True))
#   grads = tape.gradient(loss, model.trainable_variables)
#   optimizer.apply_gradients(zip(grads, model.trainable_variables))

#   return loss

In [0]:
# # Training step
# EPOCHS = 10

# for epoch in range(EPOCHS):
#   start = time.time()

#   # initializing the hidden state at the start of every epoch
#   # initally hidden is None
#   hidden = model.reset_states()

#   for (batch_n, (inp, target)) in enumerate(dataset):
#     loss = train_step(inp, target)

#     if batch_n % 100 == 0:
#       template = 'Epoch {} Batch {} Loss {}'
#       print(template.format(epoch+1, batch_n, loss))

#   # saving (checkpoint) the model every 5 epochs
#   if (epoch + 1) % 5 == 0:
#     model.save_weights(checkpoint_prefix.format(epoch=epoch))

#   print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
#   print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

# model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 9.882539749145508
Epoch 1 Loss 7.2837
Time taken for 1 epoch 23.99730157852173 sec

Epoch 2 Batch 0 Loss 7.060558319091797
Epoch 2 Loss 7.2134
Time taken for 1 epoch 22.27286386489868 sec

Epoch 3 Batch 0 Loss 7.0147294998168945
Epoch 3 Loss 7.2132
Time taken for 1 epoch 22.245380401611328 sec

Epoch 4 Batch 0 Loss 6.999016284942627
Epoch 4 Loss 7.2098
Time taken for 1 epoch 22.202122926712036 sec

Epoch 5 Batch 0 Loss 6.984915256500244
Epoch 5 Loss 7.2019
Time taken for 1 epoch 22.611298322677612 sec

Epoch 6 Batch 0 Loss 7.104455471038818
Epoch 6 Loss 7.1975
Time taken for 1 epoch 22.155707120895386 sec

Epoch 7 Batch 0 Loss 6.944394588470459
Epoch 7 Loss 7.1833
Time taken for 1 epoch 22.18338441848755 sec

Epoch 8 Batch 0 Loss 6.935486316680908
Epoch 8 Loss 7.1747
Time taken for 1 epoch 22.04549527168274 sec

Epoch 9 Batch 0 Loss 6.921158313751221
Epoch 9 Loss 7.1634
Time taken for 1 epoch 22.10192346572876 sec

Epoch 10 Batch 0 Loss 6.917452812194824
Epoch 10 L

In [0]:
model.save('/content/gdrive/My Drive/model.h5')

In [0]:
model_exp = model.get_model()
pickle.dump(model_exp, open('model_genere', 'wb'))

AttributeError: ignored